In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.color_palette("muted")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings('ignore')
import pickle as pkl
from collections import defaultdict
#%matplotlib inline

In [29]:
df = pd.read_csv('../data/fitting_data.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df = df.fillna(0)
y = df.pop('music').values
X = df.values
#X_training, X_holdout, y_training, y_holdout = train_test_split(X, y, test_size=0.20, random_state=45)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25, random_state=46,stratify=df[['african_american','latino']])






In [14]:
df = pd.read_csv('../data/fitting_data.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df = df.fillna(0)
y = df.pop('music').values
X = df.values
#X_training, X_holdout, y_training, y_holdout = train_test_split(X, y, test_size=0.20, random_state=45)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.25, random_state=46,stratify=X[:,9])


# Pipeline dictionary
pipelines = {
    'l1' : make_pipeline(StandardScaler(), LogisticRegression( penalty = 'l1', random_state=125)),
    'l2' : make_pipeline(StandardScaler(), LogisticRegression( penalty = 'l2', random_state=125)),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier(random_state=125)),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=125)),
    'linsvc' : make_pipeline(StandardScaler(), SVC(random_state=125,probability=True)),
    'rbfsvc' : make_pipeline(StandardScaler(), SVC(random_state=125,probability=True))
}

# Logistic Regression hyperparameters
l1_hyperparameters = {
    'logisticregression__C' : np.linspace(1e-3, 1e2, 1000)
}

l2_hyperparameters = {
    'logisticregression__C' : np.linspace(1e-3, 1e2, 1000)
}

# Random Forest hyperparameters
rf_hyperparameters = {
    'randomforestclassifier__n_estimators': [100, 300, 500],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 0.33],
    'randomforestclassifier__max_depth': [1, 2, 3, 4, 5]
}

# Boosted Tree hyperparameters
gb_hyperparameters = {
    'gradientboostingclassifier__n_estimators': [100, 300, 500],
    'gradientboostingclassifier__learning_rate': [0.01, 0.1, 0.5, 1],
    'gradientboostingclassifier__max_depth': [1, 2, 3, 4, 5]
}

linsvc_hyperparameters = {
    'svc__C' : [1e-5, 1e-3, 1e-1, 1e1],
    'svc__kernel' : ['linear']
}

rbfsvc_hyperparameters = {
    'svc__C': [1e-5, 1e-3, 1e-1, 1e1],
    'svc__gamma' : [1e-5, 1e-3, 1e-1, 1e1],
    'svc__kernel' : ['rbf']
}
# Create hyperparameters dictionary
hyperparameters = {
    'l1' : l1_hyperparameters, 
    'l2' : l2_hyperparameters,
    'rf' : rf_hyperparameters,
    'gb' : gb_hyperparameters,
    'linsvc' : linsvc_hyperparameters,
    'rbfsvc' : rbfsvc_hyperparameters
}
# Create data pointing dictionary
datapointers = {
    'l1' : 'logistic',
    'l2' : 'logistic',
    'rf' : 'not_logistic',
    'gb' : 'not_logistic',
    'linsvc' : 'not logistic',
    'rbfsvc' : 'not logistic'
}

# number of variables to be excluded for logistic regression
# these must be the first two variables in the feature matrix
logit_num = 4

# Create empty dictionary called fitted_models
fitted_models = {}

# Loop through model pipelines, tuning each one and saving it to fitted_models
for name, pipeline in pipelines.items():
    # Create cross-validation object from pipeline and hyperparameters
    model = GridSearchCV(pipeline, hyperparameters[name], scoring = 'neg_log_loss', cv=10, refit=True)

    # Fit model on X_train, y_train
    if datapointers[name] == 'logistic':
        model.fit(X_train[:,logit_num:], y_train)  
    else:
        model.fit(X_train, y_train)
    # Store model in fitted_models[name] 
    fitted_models[name] = model    
    
    # Print '{name} has been fitted'
    print(datapointers[name], name, 'has been fitted.')
    
for name, model in fitted_models.items():
    if datapointers[name] == 'logistic':
        pred_train = fitted_models[name].predict_proba(X_train[:,logit_num:])
        pred_test = fitted_models[name].predict_proba(X_test[:,logit_num:])
    else:
        pred_train = fitted_models[name].predict_proba(X_train)
        pred_test = fitted_models[name].predict_proba(X_test)
    # Get just the prediction for the positive class (1)
    pred_train = [p[1] for p in pred_train]
    pred_test = [p[1] for p in pred_test]
    # Calculate ROC curve
    fpr_train, tpr_train, thresholds_train = roc_curve(y_train, pred_train)
    fpr_test, tpr_test, thresholds_test = roc_curve(y_test, pred_test)
    # Calculate AUROC
    print('model', name, ': AUROC train =', auc(fpr_train, tpr_train))
    print('model', name, ': AUROC test =', auc(fpr_test, tpr_test))

print('modeling finished')

TypeError: unhashable type: 'slice'

In [ ]:
'''
multiple plots
'''

# Initialize figure
fig = plt.figure(figsize=(8,8))
plt.title('Receiver Operating Characteristic',fontsize=20)


for name, model in fitted_models.items():
    if datapointers[name] == 'logistic':
        pred = fitted_models[name].predict_proba(X_test[:,logit_num:])
    else:
        pred = fitted_models[name].predict_proba(X_test)
    pred = [p[1] for p in pred]
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    
    # Plot ROC curve
    plt.plot(fpr, tpr, label=name,linewidth=3)

# Diagonal 45 degree line
plt.plot([0,1],[0,1],'k--',linewidth=3)
plt.legend(loc='lower right',fontsize=16)

# Axes limits and labels
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.ylabel('True Positive Rate',fontsize=18)
plt.xlabel('False Positive Rate',fontsize=18)
plt.tick_params(labelsize=16)
plt.grid(True)
plt.show()
fig.savefig('../plots/ROC_full.pdf',bbox_inches='tight')

In [3]:
def errplot(x, y, yerr, **kwargs):
    ax = plt.gca()
    data = kwargs.pop("data")
    data.plot(x=x, y=y, yerr=yerr, kind="bar", ax=ax, **kwargs)

coefficients = fitted_models['l2'].best_estimator_.named_steps['logisticregression'].coef_.reshape(-1,1)[:,0]
features = np.array(df.columns)[logit_num:]
coefficients_inds = coefficients.argsort()
sorted_coefficients = coefficients[coefficients_inds[::-1]]
sorted_features = features[coefficients_inds[::-1]]
plot_colors = ['royalblue','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','darksalmon']
ax = sns.boxplot(x=sorted_coefficients,y=sorted_features,color='grey')#,palette=plot_colors)
ax.errplot
ax.set_xlabel(r'$\leftarrow$' + 'non-music major                                   music major'+r'$\rightarrow$',fontsize=13)
ax.set_ylabel('Student Characteristic',fontsize=13)
plt.title('Which graduates will continue in music?',fontsize=13)
plt.tick_params(labelsize=11)
ax.get_xaxis().set_ticks([])
fig = ax.get_figure()
fig.savefig('../plots/coefficients.pdf',bbox_inches='tight')





NameError: name 'fitted_models' is not defined

In [ ]:
coefficients = fitted_models['gb'].best_estimator_.named_steps['gradientboostingclassifier'].feature_importances_.reshape(-1,1)[:,0]
features = np.array(df.columns)
coefficients_inds = coefficients.argsort()
sorted_coefficients = coefficients[coefficients_inds[::-1]]
sorted_features = features[coefficients_inds[::-1]]
plot_colors = ['royalblue','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','darksalmon']
ax = sns.barplot(x=sorted_coefficients,y=sorted_features,color='grey')#,palette=plot_colors)
ax.set_xlabel('Characteristic Importance' +r'$\rightarrow$',fontsize=13)
ax.set_ylabel('Student Characteristic',fontsize=13)
plt.title('Which graduates will continue in music?',fontsize=13)
plt.tick_params(labelsize=11)
ax.get_xaxis().set_ticks([])
fig = ax.get_figure()
fig.savefig('../plots/featureimportance_gb.pdf',bbox_inches='tight')

In [ ]:
coefficients = fitted_models['rf'].best_estimator_.named_steps['randomforestclassifier'].feature_importances_.reshape(-1,1)[:,0]
features = np.array(df.columns)
coefficients_inds = coefficients.argsort()
sorted_coefficients = coefficients[coefficients_inds[::-1]]
sorted_features = features[coefficients_inds[::-1]]
plot_colors = ['royalblue','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey','darksalmon']
ax = sns.barplot(x=sorted_coefficients,y=sorted_features,color='grey')#,palette=plot_colors)
ax.set_xlabel('Characteristic Importance' +r'$\rightarrow$',fontsize=13)
ax.set_ylabel('Student Characteristic',fontsize=13)
plt.title('Which graduates will continue in music?',fontsize=13)
plt.tick_params(labelsize=11)
ax.get_xaxis().set_ticks([])
fig = ax.get_figure()
fig.savefig('../plots/featureimportance_rf.pdf',bbox_inches='tight')

In [30]:
'''
loop through the model to get variance in score
'''
# number of variables to be excluded for logistic regression
# these must be the first two variables in the feature matrix

# global inputs
logit_num = 4
model_names = ['l1','l2']

'''
Model Building
'''

# Pipeline dictionary
pipelines = {
    'l1' : make_pipeline(StandardScaler(), LogisticRegression( penalty = 'l1', random_state=125)),
    'l2' : make_pipeline(StandardScaler(), LogisticRegression( penalty = 'l2', random_state=125)),
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier(random_state=125)),
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=125)),
    'linsvc' : make_pipeline(StandardScaler(), SVC(random_state=125,probability=True)),
    'rbfsvc' : make_pipeline(StandardScaler(), SVC(random_state=125,probability=True))
}

# Logistic Regression hyperparameters
l1_hyperparameters = {
    'logisticregression__C' : np.linspace(1e-2, 1e1, 200)
}

l2_hyperparameters = {
    'logisticregression__C' : np.linspace(1e-2, 1e1, 200)
}

# Random Forest hyperparameters
rf_hyperparameters = {
    'randomforestclassifier__n_estimators': [100, 300, 500],
    'randomforestclassifier__max_features': ['auto', 'sqrt', 0.33],
    'randomforestclassifier__max_depth': [1, 2, 3, 4, 5]
}

# Boosted Tree hyperparameters
gb_hyperparameters = {
    'gradientboostingclassifier__n_estimators': [100, 300, 500],
    'gradientboostingclassifier__learning_rate': [0.01, 0.1, 0.5, 1],
    'gradientboostingclassifier__max_depth': [1, 2, 3, 4, 5]
}

linsvc_hyperparameters = {
    'svc__C' : [1e-5, 1e-3, 1e-1, 1e1],
    'svc__kernel' : ['linear']
}

rbfsvc_hyperparameters = {
    'svc__C': [1e-5, 1e-3, 1e-1, 1e1],
    'svc__gamma' : [1e-5, 1e-3, 1e-1, 1e1],
    'svc__kernel' : ['rbf']
}
# Create hyperparameters dictionary
hyperparameters = {
    'l1' : l1_hyperparameters, 
    'l2' : l2_hyperparameters,
    'rf' : rf_hyperparameters,
    'gb' : gb_hyperparameters,
    'linsvc' : linsvc_hyperparameters,
    'rbfsvc' : rbfsvc_hyperparameters
}
# Create data pointing dictionary
datapointers = {
    'l1' : 'logistic',
    'l2' : 'logistic',
    'rf' : 'not_logistic',
    'gb' : 'not_logistic',
    'linsvc' : 'not logistic',
    'rbfsvc' : 'not logistic'
}

def model_scoring_auc(X_in, y_in, model, datapointer):
    if datapointer == 'logistic':
        pred = model.predict_proba(X_in[:,logit_num:])
    else:
        pred = model.predict_proba(X_in)
    # Get just the prediction for the positive class (1)
    pred = [p[1] for p in pred]
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_in, pred)
    # Calculate AUROC
    return auc(fpr, tpr)


def model_fitting(X, y, logit_num, model_names, pipelines, hyperparameters, datapointers, randstate):
    # Create empty dictionary called fitted_models
    fitted_models = {}
    fitted_scores = {}
    
    # split data for CV testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=randstate,stratify=X[:,9])

    # Loop through model pipelines, tuning each one and saving it to fitted_models
    for name in model_names:
        # Create cross-validation object from pipeline and hyperparameters
        model = GridSearchCV(pipelines[name], hyperparameters[name], scoring = 'neg_log_loss', cv=10, refit=True)

        # Fit model on X_train, y_train
        if datapointers[name] == 'logistic':
            model.fit(X_train[:,logit_num:], y_train)  
        else:
            model.fit(X_train, y_train)
        # Store model in fitted_models[name] 
        fitted_models[name] = model
        
        # store scores in fitted_scores[name]
        train_score = model_scoring_auc(X_train, y_train, model, datapointers[name])
        test_score = model_scoring_auc(X_test, y_test, model, datapointers[name])
        fitted_scores[name] = [train_score,test_score]
            
    return fitted_models, fitted_scores

        


In [32]:

df = pd.read_csv('../data/fitting_data.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df = df.fillna(0)
y = df.pop('music').values
X = df.values


models_iterate = {}
scores_iterate = {}
for i in range(10):
    models_iterate[i], scores_iterate[i] = model_fitting(X,y,logit_num,model_names,pipelines,hyperparameters,datapointers,1000+i)
    print(scores_iterate[i])

KeyboardInterrupt: 

In [13]:
models_iterate[1]['l1'].best_estimator_.named_steps['logisticregression'].coef_

array([[-0.56049574,  0.48019651,  0.00307966,  0.38785114,  0.21788662,
         0.0359901 , -0.3606645 , -0.01032148, -0.26202951,  0.3070357 ,
        -0.25403004]])

In [19]:
df['african_american']

KeyError: ('african_american', 'latino')

In [24]:
X_train.shape

(355, 15)

In [25]:
X_test.shape

(119, 15)

In [26]:
355+119

474

In [28]:
X.shape

(474, 15)